#Traversing Mountain Project

In [ ]:
def is_climb(page):
    # If you a span with class="rateYDS" inside an h3 header, then the page is a climb
    # Alternative, on a climb page you will be able to find the words "You & This Route" inside a div with ...
    # id="youContainer" whereas on an area page you will instead be able ot find the words "You & This Area"
    return bool

In [ ]:
def get_all_links_down_hierarchy(page):
    # The links are found inside of div with id="viewerLeftNavColContent"
    # The first div inside this div should have id="mpbox########" where #### will be different for every page
    # Every link in id="mpbox########" should be a link going further down the hierarchy with the exception:
        # The link associated with <img src="/img/up.gif" .../> should be excluded
        # Any links to mountainproject.com/scripts/... should be excluded
    return list_of_links

In [ ]:
def scrape(url):
    # Beautiful soup (or whatever) code here to get page from url
    # Ideally, we will do things such that we pass this data on and don't request the link again to ...
    # exact other information from the page
    return page

In [ ]:
def get_number_of_ratings(page):
    # Each review on a climb page is in its own table of class="comvis"
    # Inside each table are 2 <td> cells; everything we need is in the first; probably best to code this in ...
    # because the second is user input and could have weird stuff that throws us off.
    # We can get the link to the user by looking for href="/u/...".
    # We can get where the user rated the route by see if the <img> with ... 
    # src="http://www.mountainproject.com/img/stars/star_b.gif" is present in the table.
    return number_of_ratings

In [ ]:
def extract_climb_info(page):
    # Will be copied and adapted from "Climb Page Scraping.ipynb"
    return climb_info

In [ ]:
def extract_links_to_user_pages(page):
    # The links from get_number_of_ratings above.  *We only want the links to users who actually rated the
    # route and didn't just submit a comment with no rating.
    return list_of_links

In [ ]:
def extract_user_info(url):
    # Will be copied and adapted from "User Page Scraping.ipynb"
    return user_info

In [ ]:
# Let's do the traversing recursively

def traverse(url, min_climb_ratings, climbs_info, user_links):
    page = scrape(url)
    
    if !is_climb(page):
        links = get_all_links_down_hierarchy(page)
        for link in links:
            traverse(link, min_climb_ratings)
    else:
        num_ratings = get_number_of_ratings(page)
        if num_ratings >= min_climb_ratings
            climbs_info.append(extract_climb_info(page)) # or .update() if we want climbs_info to be a dict
            user_links.extend([(link, 1) for link in extract_links_to_user_pages(page)])

min_climb_ratings = 10
min_user_ratings = 10
            
climbs_info = [] # or {} if we want climbs_info to be a dict
user_links = []
top_page = scrape("http://www.mountainproject.com/destinations/")
state_links = get_all_links_to_state_pages(top_page)
for link in state_links:
    traverse(link, min_climb_ratings)
    
user_links_reduced = map_reduce(user_links) # user_links is now a list of tuples (link_to_user, num_ratings)

user_links_filtered = [user[0] for user in user_links_filtered if user[1] >= min_user_ratings]

users_info = [] # or {} if we want users_info to be a dict
for link in user_links_filtered:
    users_info.append(extract_user_info(link))

In [ ]:
"http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=10&selectedIds=105872225&type=mixed&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=11900&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating
"

In [ ]:
"http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=10&selectedIds=105872225"

In [ ]:
# Get area id's from: http://www.mountainproject.com/destinations/
# My code starts below

In [1]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests

In [3]:
nh_mixed = requests.get("http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=10&selectedIds=%27105872225&type=mixed&diffMinrock=1800&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=5500&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=quality_stars+desc")

In [5]:
soup = BeautifulSoup(nh_mixed.text, "html.parser")

In [49]:
tables = soup.find_all("table", attrs = {"class" : "objectList"})
entries = tables[1].find_all("tr")
test = [x.find_all("td") for x in entries]

In [57]:
result = [x[0].find("a").get("href") for x in test[1:]]

In [58]:
result

[u'/v/pegasus-rock-finish/106106926',
 u'/v/black-dike/105890633',
 u'/v/repentance/105872936',
 u'/v/greens-chasm/106680167']

In [53]:
#cleaner = lambda r: {'Route Name': r[0].get_text(), 'Route URL': r[0].find("a").get("href")}